In [13]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision.models import resnet18
from torchvision.datasets import ImageFolder
from torchvision.transforms import ToTensor
import numpy as np
from sklearn.metrics import classification_report, roc_auc_score

In [2]:
train_dataset = ImageFolder("/kaggle/input/cv-dataset-2/data_folder/train",transform=ToTensor())
test_dataset = ImageFolder("/kaggle/input/cv-dataset-2/data_folder/test",transform=ToTensor())

In [3]:
train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=64)

In [4]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = resnet18(num_classes=2).to(device)
value_to_prob = nn.Softmax(dim=1).to(device)
opt = optim.Adam(model.parameters(),lr=1e-4)
loss_fn = nn.CrossEntropyLoss()
min_loss = np.inf
for epoch in range(100):
    train_losses = []
    for X, Y in train_dataloader:
        X = X.to(device)
        Y = Y.to(device)
        opt.zero_grad()
        prob = value_to_prob(model(X))
        loss = loss_fn(prob, Y)
        loss.backward()
        opt.step()
        train_losses.append(loss.item())
    test_losses = []
    with torch.no_grad():
        for X, Y in test_dataloader:
            X = X.to(device)
            Y = Y.to(device)
            prob = value_to_prob(model(X))
            loss = loss_fn(prob, Y)
            test_losses.append(loss.item())
    train_loss = np.mean(train_losses)
    test_loss = np.mean(test_losses)
    print(f"epoch {epoch}/100\ttrain : {train_loss:.6f}\ttest : {test_loss:.6f}")
    if test_loss < min_loss:
        min_loss = test_loss
        torch.save(model.state_dict(),"best_model.pt")

epoch 0/100	train : 0.401383	test : 0.714335
epoch 1/100	train : 0.330702	test : 0.729753
epoch 2/100	train : 0.319732	test : 0.694247
epoch 3/100	train : 0.316455	test : 0.701846
epoch 4/100	train : 0.316426	test : 0.695879
epoch 5/100	train : 0.315218	test : 0.698672
epoch 6/100	train : 0.315414	test : 0.693790
epoch 7/100	train : 0.314937	test : 0.689087
epoch 8/100	train : 0.315516	test : 0.689351
epoch 9/100	train : 0.315921	test : 0.678612
epoch 10/100	train : 0.315329	test : 0.679529
epoch 11/100	train : 0.314978	test : 0.670869
epoch 12/100	train : 0.314924	test : 0.659261
epoch 13/100	train : 0.314756	test : 0.672492
epoch 14/100	train : 0.314577	test : 0.638569
epoch 15/100	train : 0.314459	test : 0.648977
epoch 16/100	train : 0.314494	test : 0.656884
epoch 17/100	train : 0.314199	test : 0.636348
epoch 18/100	train : 0.313658	test : 0.642864
epoch 19/100	train : 0.314472	test : 0.635452
epoch 20/100	train : 0.313977	test : 0.644575
epoch 21/100	train : 0.313823	test : 0.64117

In [6]:
true_y = []
prob_y = []
with torch.no_grad():
    for X, Y in test_dataloader:
        X = X.to(device)
        prob = value_to_prob(model(X))
        true_y.append(Y.numpy())
        prob_y.append(prob.cpu().numpy())

In [9]:
prob_y = np.concatenate(prob_y)

In [10]:
true_y = np.concatenate(true_y)

In [11]:
pred_y = np.argmax(prob_y,axis=1)

In [12]:
pred_y

array([0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0,
       1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0,
       1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0,
       0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0,
       1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1,
       0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1,
       0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1,

In [14]:
print(classification_report(true_y,pred_y))

              precision    recall  f1-score   support

           0       0.81      0.77      0.79       500
           1       0.78      0.82      0.80       500

    accuracy                           0.79      1000
   macro avg       0.79      0.79      0.79      1000
weighted avg       0.79      0.79      0.79      1000



In [16]:
roc_auc_score(true_y,prob_y[:,1])

0.784052